In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-06-23


Found rundate 2023-06-22 10:00:02 < 2023-06-23 at commit bd81905e; returning commit 40e618d


'2023-06-23 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-06-23 10:00:02, NJSP was reporting 247 YTD deaths
Current YTD Deaths (2024-06-23 10:00:03-04:00): 304
Previous year YTD Deaths (adjusted; 2023-06-23 10:00:02): 247.74591789368847
Projected 2024 total: 704.3348888550443
22.7% change


FAUQStats cache miss: 2023, Fri Jun 23 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,Mulberry St,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,NaN,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,322,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,South Randolphville Rd,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,Outwater Ln,Outwater Ln,1.0,0.0,0.0,1.0,0.0,NaN,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,Washington St,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12517,03,Burlington,0325,Mount Laurel Twsp,NaN,Interstate 295 MP 37.2,1.0,1.0,0.0,0.0,0.0,295,2.0,2023-06-19 21:06:00-04:00
12514,08,Gloucester,0818,Washington Twsp,NaN,County 655,1.0,1.0,0.0,0.0,0.0,655,NaN,2023-06-20 05:08:00-04:00
12518,03,Burlington,0335,Springfield Twsp,NaN,State Highway 206,1.0,1.0,0.0,0.0,0.0,206,1.0,2023-06-20 16:54:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,14,1,11,2,28
Bergen,9,3,9,0,20
Burlington,16,3,2,1,22
Camden,11,3,10,0,24
Cape May,1,0,0,0,1
Cumberland,8,2,2,0,11
Essex,8,3,11,0,22
Gloucester,7,2,0,0,9
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

47.6% through the year, 52.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,8,18,14
Bergen,driver,9,20,9
Burlington,driver,7,26,16
Camden,driver,9,18,11
Cape May,driver,2,4,1
...,...,...,...,...
Salem,crashes,4,11,7
Somerset,crashes,5,21,6
Sussex,crashes,1,6,3


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,62,4,28,7,24
Bergen,41,1,20,9,15
Burlington,60,3,47,6,4
Camden,56,3,21,7,27
Cape May,4,0,3,1,1
Cumberland,25,0,21,5,3
Essex,62,1,20,7,36
Gloucester,22,1,15,4,2
Hudson,12,1,6,5,3


In [9]:
projected.to_csv(PROJECTED_CSV)